# IVF_PQ
- IVF_PQ 인덱스는 고차원 공간에서 대략적인 최근접 이웃 검색을 위한 양자화 기반 인덱싱 알고리즘
- 일부 그래프 기반 방법만큼 빠르진 않지만 IVF_PQ는 메모리를 훨씬 적게 필요로 하므로 대규모 데이터 세트에 실용적인 선택

참고자료: https://milvus.io/docs/ko/v2.5.x/ivf-pq.md

## 개요
- IVF_PQ는 효율적인 벡터 검색과 검색을 위해 인덱싱과 압축을 결합한 하이브리드 접근 방식인 IVF와 PQ를 결합

### IVF
- 책에서 색인을 만드는 것과 같음
- 모든 페이지를 스캔하는 대신 색인에서 특정 키워드를 조회하여 관련 페이지를 빠르게 찾음
- 벡터가 클러스터로 그룹화되고 알고리즘이 쿼리 벡터에 가까운 몇 개의 클러스터 내에서 검색함
- 작동방식
    1. 클러스터링: 벡터 데이터 세트는 k-means와 같은 클러스터링 알고리즘을 사용하여 지정된 수의 클러스토러 나뉨
    2. 할당: 각 벡터는 중심이 가장 가까운 클러스터에 할당
    3. 반전 인덱스: 각 클러스터 중심으로 해당 클러스터에 할당된 벡터 목록에 맵핑하는 인덱스가 생성
    4. 검색: 가장 가까운 이웃을 검색할 때 검색 알고리즘은 쿼리 벡터와 클러스터 중심을 비교하여 가장 가능성이 높은 클러스터를 선택
    

## 인덱스 구축
- Milvus의 벡터 필드에 IVF_PQ 인덱스를 구축하려면 add_index() 방법을 사용하여 index_type, metric_type 및 인덱스에 대한 추가 매개 변수를 지정

In [1]:
from pymilvus import MilvusClient

index_parmas = MilvusClient.prepare_index_params()

index_parmas.add_index(
    field_name="vector",
    index_type="IVF_PQ",
    index_name="vector_index",
    metric_type="L2",
    params={
        "m":4, # 벡터를 분할하기 위한 하위 벡터의 수
    }
)

## 인덱스에서 검색

In [3]:
search_params = {
    "params": {
        "nprobe": 10,
    }
}

client = MilvusClient(
    uri="http://localhost:19530"
)

res = client.search(
    collection_name="my_collection",
    anns_field="vector",
    data=[[0.1, 0.2, 0.3, 0.4, 0.5]],
    limit=3,
    search_params=search_params
)